In [4]:
import os
import random
import shutil
import Augmentor

In [ ]:
# single classifier

version_num = 1
train_size = 500
valid_size = 50
dest_dir = f"data/single_classifier_data_v{version_num}"

# create directories
for directory in ["train", "valid", "test"]:
    for pothole_type in ["mild", "moderate", "severe"]:
        if not os.path.exists(f"./{dest_dir}/{directory}/{pothole_type}"):
            os.makedirs(f"./{dest_dir}/{directory}/{pothole_type}")
            
# make train, valid, test sets
for pothole_type in ["mild", "moderate", "severe"]:
    for directory in ["train", "valid", "test"]:
        source_dir_name = f"data/raw/{pothole_type}/{key}"
        l = os.listdir(source_dir_name)

        for i, file in enumerate(l):
            path = os.path.join(source_dir_name, file)
            shutil.copyfile(path, f"./{dest_dir}/{directory}/{pothole_type}")
                
# augmentation
for directory in ["train", "valid"]:
    for pothole_type in ["mild", "moderate", "severe"]:
        for label in ["0", "1"]:
            source_dir = f"./{dest_dir}/{pothole_type}/{directory}/{label}"
            aug_dir = f"./{dest_dir}/{pothole_type}/{directory}/{label}_aug"
            os.makedirs(aug_dir)
            
            sample_num = train_size if directory == "train" else valid_size
            p = Augmentor.Pipeline(source_directory=source_dir, output_directory=f"../{label}_aug")
            p.flip_left_right(probability=0.5)
            p.rotate(probability=0.8, max_left_rotation=10, max_right_rotation=10)
            p.zoom(probability=0.5, min_factor=1.1, max_factor=1.4)
            p.sample(sample_num)
            
            shutil.rmtree(source_dir)
            os.rename(aug_dir, source_dir)

In [3]:
# one vs all classifier

version_num = 1
dest_dir = f"data/one_vs_all_classifier_data_v{version_num}"

# create directories 
for pothole_type in ["mild", "moderate", "severe"]:
    for directory in ["train", "valid", "test"]:
        if not os.path.exists(f"./{dest_dir}/{pothole_type}/{directory}"):
            os.makedirs(f"./{dest_dir}/{pothole_type}/{directory}/0")
            os.makedirs(f"./{dest_dir}/{pothole_type}/{directory}/1")

# split into train, valid, test
for directory in ["mild", "moderate", "severe"]:
    for pothole_type in ["mild", "moderate", "severe"]:
        for key in ["train", "valid", "test"]:
            source_dir_name = f"data/raw/{pothole_type}/{key}"
            l = os.listdir(source_dir_name)
            label = str(0) if pothole_type == directory else str(1)
            
            for i, file in enumerate(l):
                path = os.path.join(source_dir_name, file)
                shutil.copyfile(path, os.path.join(dest_dir, directory, key, label, file))

# augmentation
for pothole_type in ["mild", "moderate", "severe"]:
    for directory in ["train", "valid"]:
        for label in ["0", "1"]:
            source_dir = f"./{dest_dir}/{pothole_type}/{directory}/{label}"
            aug_dir = f"./{dest_dir}/{pothole_type}/{directory}/{label}_aug"
            os.makedirs(aug_dir)
            
            sample_num = train_size if directory == "train" else valid_size
            p = Augmentor.Pipeline(source_directory=source_dir, output_directory=f"../{label}_aug")
            p.flip_left_right(probability=0.5)
            p.rotate(probability=0.8, max_left_rotation=10, max_right_rotation=10)
            p.zoom(probability=0.5, min_factor=1.1, max_factor=1.4)
            p.sample(sample_num)
            
            shutil.rmtree(source_dir)
            os.rename(aug_dir, source_dir)